In [10]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, FieldCondition, PointStruct
import requests

client = QdrantClient(url="http://localhost:6333")

In [11]:
client.create_collection(
    collection_name="test_collection",
    vectors_config=VectorParams(size=4096, distance=Distance.COSINE)
)

True

In [12]:
data = {
    "texts": ["人工智慧很有趣"],
    "normalize": True,
    "batch_size": 32
}

response = requests.post("http://ws-04.wade0426.me/embed", json=data)

print(f"Status:, {response.status_code}")
print(f"Response: {response.text}")

if response.status_code == 200:
    result =response.json()
    print(f"dim: {result['dimension']}")
else:
    print(f"Error: {response.json()}")

Status:, 200
Response: {"embeddings":[[0.021484375,0.0012845993041992188,0.0196990966796875,-0.034210205078125,0.04559326171875,-0.0269012451171875,-0.01934814453125,-0.01177215576171875,-0.0158843994140625,0.03338623046875,-0.026397705078125,0.02337646484375,0.052642822265625,0.01349639892578125,0.0270233154296875,0.0168609619140625,-0.033477783203125,-0.0208282470703125,0.026275634765625,0.004146575927734375,-0.016571044921875,0.0305633544921875,0.0184326171875,-0.0004763603210449219,-0.01439666748046875,0.026397705078125,-0.0188751220703125,-0.0124359130859375,-0.000518798828125,0.02099609375,0.0041656494140625,-0.023162841796875,-0.006610870361328125,0.02166748046875,0.0125579833984375,0.038543701171875,-0.00975799560546875,-0.002780914306640625,0.01016998291015625,0.0082244873046875,0.018218994140625,-0.0205841064453125,0.026214599609375,-0.01070404052734375,-0.006946563720703125,-0.01419830322265625,-0.0335693359375,-0.021270751953125,-0.01390838623046875,0.0295867919921875,0.013

In [13]:
client.upsert(
    collection_name="test_collection",
    points=[
        PointStruct(
            id=1,
            vector=result['embeddings'][0],
            payload={"text":"人工智慧很有趣", "metadata": "其他資訊"}
        )
    ]
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [14]:
def get_embedding(texts):
    # 1. 如果輸入是單一字串，轉成列表
    if isinstance(texts, str):
        texts = [texts]

    # 2. 準備呼叫 API 的資料
    # 注意：這裡必須跟上方存入資料時的參數一致 (normalize=True)
    data = {
        "texts": texts,
        "normalize": True,
        "batch_size": 32
    }

    try:
        # 3. 呼叫你的 Embedding API
        response = requests.post("http://ws-04.wade0426.me/embed", json=data)
        response.raise_for_status() # 檢查 HTTP 是否成功

        result = response.json()

        # 4. 回傳 embedding 列表
        return result['embeddings']

    except Exception as e:
        print(f"Embedding API 呼叫失敗: {e}")
        return []

In [15]:
from qdrant_client.models import Filter, FieldCondition, Range

texts = ["AI 有什麼好處？"]
embeddings = get_embedding(texts)

if embeddings:
    query_vector = embeddings[0]

    print("\n=== 一般搜尋結果 ===")
    search_result = client.query_points(
        collection_name="test_collection",
        query=query_vector,
        limit=3
    )

    for point in search_result.points:
        print(f"ID: {point.id}")
        print(f"相似度: {point.score:.4f}") 
        print(f"內容: {point.payload.get('text')}")
        print("---")

    # --- 帶有過濾器的搜尋 ---
    print("\n=== 帶過濾器的搜尋結果 ===")
    results = client.query_points(
        collection_name="test_collection",
        query=query_vector,
        query_filter=Filter(
            must=[
                FieldCondition(
                    key="year",
                    range=Range(
                        gte=3,
                        lte=10
                    )
                )
            ]
        ),
        limit=3
    )

    if not results.points:
        print("沒有符合 year 條件的資料")

    for point in results.points:
        print(f"ID: {point.id}")
        print(f"Payload: {point.payload}")
        print("-" * 30)


=== 一般搜尋結果 ===
ID: 1
相似度: 0.7040
內容: 人工智慧很有趣
---

=== 帶過濾器的搜尋結果 ===
沒有符合 year 條件的資料
